In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import statsmodels.api as sm
from sklearn import linear_model
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error
from statsmodels.regression.quantile_regression import QuantReg

from sklearn.preprocessing import StandardScaler

import os

from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv('./ml-1m/movies.dat',sep='::')

c:\users\epsilon\dev\anaconda\envs\fortf\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
rt = np.loadtxt('./ml-1m/movies.dat',delimiter='::',dtype=np.int64)

#### - UserIDs  1 ~ 6040 
#### - MovieIDs 1 ~ 3952
#### - Ratings  5-star scale (whole-star ratings only)
#### - Timestamp is represented in seconds since the epoch as returned by time(2)

- rating mean

In [6]:
rt[:,2].mean()

3.581564453029317

- How many users participated

In [7]:
len(rt)

1000209

In [8]:
users = len(np.unique(rt[:,0]))
users

6040

#### - mean of rating for each user

##### using for 

In [9]:
user_mean =[]
for x in range(users):
    rt_ = rt[:,0] == (x+1)
    if x ==0:
        print(rt_)
    user = rt[rt_,2]
    user_mean.append(user.mean())

[ True  True  True ... False False False]


In [10]:
np.mean(user_mean)

3.702704866999724

- transform to Pandas 

In [11]:
rtp = pd.DataFrame(rt,columns=['usr_id','mv_id','star','timestamp'])

In [12]:
rtp['star_mean'] = rtp.groupby(['usr_id'])['star'].transform('mean')
rtp['star_std'] = rtp.groupby(['usr_id'])['star'].transform('std')

In [13]:
for qt in [0.0,0.25,0.5,0.75,1] :
    rtp['star_qt%d'%qt] = rtp.groupby(['usr_id'])['star'].transform(lambda x: x.quantile(q=qt))

#### integrate Data (movies.dat,ratings.dat,users.dat)
- "ratings.dat" UserID::MovieID::Rating::Timestamp
- "users.dat"   UserID::Gender::Age::Occupation::Zip-code
- "movies.dat"  MovieID::Title::Genres
> integrated data :  
UserID::UserID::Gender::Age::Occupation::Zip-code::
MovieID::MovieID::Title::Genres
::Rating
::Timestamp 

In [23]:
mv  = pd.read_csv('ml-1m/movies.dat',sep='::',names=['mov_id','title','genres'])
usr = pd.read_csv('ml-1m/users.dat' ,sep='::',names=['usr_id','gender','age','occupation','zip-code'])

c:\users\epsilon\dev\anaconda\envs\fortf\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
c:\users\epsilon\dev\anaconda\envs\fortf\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [145]:
usr = usr.set_index('usr_id')

In [146]:
mv  = mv.set_index('mov_id')

In [141]:
usr_info = rtp['usr_id'].transform([
                        lambda x : usr.loc[x]['gender'] ,
                        lambda x : usr.loc[x]['age'] ,
                        lambda x : usr.loc[x]['occupation'] ,
                        lambda x : usr.loc[x]['zip-code']                        
                       ])
usr_info.columns = ['gender','age','occupation','zip-code']

mv_info = rtp['mv_id'].transform([
                        lambda x : usr.loc[x]['Title'] ,
                        lambda x : usr.loc[x]['Genres']                
                       ])
usr_info.columns = ['gender','age','occupation','zip-code']
mv_info .columns = ['Title','Genres']


,<lambda>,<lambda>,<lambda>,<lambda>
0,F,1,10,48067
1,F,1,10,48067
2,F,1,10,48067
3,F,1,10,48067
4,F,1,10,48067
...,...,...,...,...
1000204,M,25,6,11106
1000205,M,25,6,11106
1000206,M,25,6,11106
1000207,M,25,6,11106
